# Utilizando raspagem das páginas

Este código cria três tabelas em um banco de dados SQLite, organizando as informações da seguinte forma:

- researchers: Armazena o ID e o nome de cada pesquisador.
- academic_titles: Registra os detalhes de cada título acadêmico, vinculando-os a um pesquisador por meio do seu ID.
- advisors_academic_titles: Mapeia a relação entre orientadores e títulos acadêmicos, que, por sua vez, estão associados a pesquisadores.

Outra fonte:
https://github.com/davidalber/geneagrapher

In [ ]:
# Bibliotecas
from bs4 import BeautifulSoup
import requests
import re
import sqlite3
import pandas as pd
import time
from tqdm import tqdm

ip = requests.get('https://api64.ipify.org?format=text').text
print("IP externo dessa sessão:", ip)

# Caminho onde o banco está/será salvo
db_sqlite = "./data/mgp.sqlite"
db_mgp_optimized = "./data/mgp_optimized.feather"

In [ ]:
# Objeto pesquisador
class researcher:
  def __init__(self, researcher_id, name, academic_titles, descendants_id):
    self.researcher_id = researcher_id
    self.name = name
    self.academic_titles = academic_titles
    self.descendants_id = descendants_id

  def __str__(self):
    return f'{self.researcher_id} - {self.name} - {self.academic_titles} - {self.descendants_id}'

  def to_list(self):
    academic_titles_list = [academic_title.to_list() for academic_title in self.academic_titles]
    return [self.researcher_id, self.name, academic_titles_list, self.descendants_id]

  def save_to_db(self, db_sqlite):
    with sqlite3.connect(db_sqlite) as conn:
      cursor = conn.cursor()
      conn = sqlite3.connect(db_sqlite)
      cursor = conn.cursor()

      # Cria as tabelas se não existir
      cursor.execute('''
        CREATE TABLE IF NOT EXISTS researchers (
          researcher_id INTEGER PRIMARY KEY,
          name TEXT NOT NULL
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS academic_titles (
          academic_title_id INTEGER PRIMARY KEY,
          researcher_id INTEGER NOT NULL,
          title TEXT NOT NULL,
          dissertation TEXT,
          institution TEXT,
          year INTEGER,
          country TEXT,
          FOREIGN KEY (researcher_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE
        )
      ''')

      cursor.execute('''
        CREATE TABLE IF NOT EXISTS advisors_academic_titles (
          advisor_id INTEGER NOT NULL,
          academic_title_id INTEGER NOT NULL,
          PRIMARY KEY (advisor_id, academic_title_id),
          FOREIGN KEY (advisor_id) REFERENCES researchers (researcher_id) ON DELETE CASCADE,
          FOREIGN KEY (academic_title_id) REFERENCES academic_titles (academic_title_id) ON DELETE CASCADE
        )
      ''')

      # Inserindo os dados
      cursor.execute('''
        INSERT INTO researchers (researcher_id, name)
        VALUES (?, ?)
      ''', (self.researcher_id, self.name))

      for academic_title in self.academic_titles:
        cursor.execute('''
          INSERT INTO academic_titles (researcher_id, title, dissertation, institution, year, country)
          VALUES (?, ?, ?, ?, ?, ?)
        ''', (self.researcher_id, academic_title.title, academic_title.dissertation, academic_title.institution, academic_title.year, academic_title.country))
        academic_title_id = cursor.lastrowid
        if isinstance(academic_title.advisors_id, (list, tuple)):
          for advisor_id in academic_title.advisors_id:
            cursor.execute('''
              INSERT INTO advisors_academic_titles (advisor_id, academic_title_id)
              VALUES (?, ?)
            ''', (advisor_id, academic_title_id))

      conn.commit()

  def close_db(db_sqlite):
    conn = sqlite3.connect(db_sqlite)
    conn.close()

# Objeto título acadêmico
class academic_title:
  def __init__(self, title, dissertation, institution, year, country, advisors_id):
    self.title = title
    self.dissertation = dissertation
    self.institution = institution
    self.year = year
    self.country = country
    self.advisors_id = advisors_id

  def __str__(self):
    return f'{self.title} - {self.dissertation} - {self.institution} - {self.year} - {self.country} - {self.advisors_id}'

  def to_list(self):
    return [self.title, self.dissertation, self.institution, self.year, self.country, self.advisors_id]

In [ ]:
# Função para usar query e parâmetros para buscar no banco
def search_db(db_sqlite, query, params=()):
  conn = sqlite3.connect(db_sqlite)
  cursor = conn.cursor()

  cursor.execute(query, params)
  results = cursor.fetchall()

  conn.close()

  return results

def get_advisor_and_student_ids(researcher_id):
  '''Essa função retorna a lista de orientadores e alunos de um pesquisador'''

  # Busca todos os orientadores (advisor_id) que estão associados a títulos acadêmicos pertencentes a um pesquisador específico (researcher_id).
  advisor_ids = search_db(db_sqlite,
                          "SELECT aat.advisor_id\
                           FROM advisors_academic_titles aat\
                           JOIN academic_titles at ON aat.academic_title_id = at.academic_title_id\
                           WHERE at.researcher_id = ?", (researcher_id,))

  # Busca os IDs dos pesquisadores que têm pelo menos um título acadêmico orientado pelo pesquisador especificado (researcher_id)
  student_ids = search_db(db_sqlite,
                          "SELECT r.researcher_id\
                           FROM researchers r\
                           JOIN academic_titles at ON r.researcher_id = at.researcher_id\
                           JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id\
                           WHERE aat.advisor_id = ?", (researcher_id,))

  # Corrige a lista
  advisor_ids = [row[0] for row in advisor_ids]
  student_ids = [row[0] for row in student_ids]

  return advisor_ids, student_ids

# Função para realizar a raspagem
def get_researcher(researcher_id):
  requestsHTML = requests.get(f'https://genealogy.math.ndsu.nodak.edu/id.php?id={researcher_id}')
  soup = BeautifulSoup(requestsHTML.text, 'html.parser')

  # Caso o ID passado não exista no site
  for dataPag in soup.find_all("p"):
    if dataPag.text == "You have specified an ID that does not exist in the database. Please back up and try again.":
      return 'ID not found'
    else:
      pass

  for dataPag in soup.find_all("div", attrs={"id": "paddingWrapper"}):
    # Encontrando dados dos descendentes
    students_id = []
    try:
      for line_students in dataPag.table.find_all("tr")[1:]:  # [1:] pula o título
        student_data = list(line_students.strings)
        students_id.append(re.search(r'id=(\d+)',line_students.a['href']).group(1))
    except:
      pass

    # Monta o objeto do titulo acadêmico e pesquisador
    academic_titles = []
    dissertations = {}
    advisor_ids = {}
    countries = {}

    # Pegando os títulos acadêmicos (grau, universidade, ano)
    for index, div in enumerate(dataPag.find_all("div", style="line-height: 30px; text-align: center; margin-bottom: 1ex")):
      spans = list(div.stripped_strings)
      country_img = div.find("img")  # Procura pela imagem da bandeira
      country = country_img["title"] if country_img and "title" in country_img.attrs else None
      if spans:
        academic_titles.append(spans)
        dissertations[index] = None  # Inicializa sem dissertação
        advisor_ids[index] = None  # Inicializa sem orientadores
        countries[index] = country

    # Pegando as dissertações corretamente e associando pelo índice
    diss_index = 0
    for div in dataPag.find_all("div", style="text-align: center"):
      spans = list(div.stripped_strings)
      if spans and "Dissertation:" in spans[0]:  # Verifica se contém "Dissertation:"
        dissertation_title = spans[1] if len(spans) > 1 else None
        if diss_index < len(academic_titles):  # Garante que não ultrapasse o número de títulos
          dissertations[diss_index] = dissertation_title
        diss_index += 1  # Passa para a próxima dissertação

    # Pegando os IDs dos orientadores e associando corretamente
    adv_index = 0
    for p in dataPag.find_all("p", style=True):  # Encontra todos os <p> com style definido
      if "line-height: 2.75ex" in p["style"]:  # Caso onde há orientadores com <a href>
        ids = [re.search(r'id=(\d+)', a['href']).group(1) for a in p.find_all("a") if a.has_attr('href')]

        if adv_index < len(academic_titles):  # Garante alinhamento com títulos acadêmicos
          advisor_ids[adv_index] = ids if ids else None

        adv_index += 1  # Passa para o próximo título acadêmico

      elif "Advisor: Unknown" in p.get_text():  # Se for um <p> indicando "Unknown"
        if adv_index < len(academic_titles):
          advisor_ids[adv_index] = None  # Marca como sem orientador

        adv_index += 1  # Passa para o próximo título acadêmico

    # Monta lista com objetos de título acadêmicos
    academic_title_data_list = []
    for index, title in enumerate(academic_titles):
      dissertation = dissertations.get(index, None)
      advisors = advisor_ids.get(index, [])
      country = countries.get(index, None)

      academic_title_data_list.append(
          academic_title(
              title[0],       #título
              dissertation,   #Dissertação
              title[1] if len(title) > 1 else None, #Instituição
              title[2] if len(title) > 2 else None, #Ano
              country,        #País
              advisors        #Lista de orientadores
          )
          )

    # Cria o objeto pesquisado
    researcher_data = researcher(
        researcher_id,
        ' '.join(dataPag.h2.text.split()),
        academic_title_data_list,
        students_id
        )

  return researcher_data

In [ ]:
# Raspar vários pesquisadores
try:
  id_author_start = search_db(db_sqlite, 'SELECT MAX(researcher_id) FROM researchers;')[0][0] + 1
except:
  id_author_start = 1
id_author_end = 10
#id_author_end = 330000

#researcher.close_db(db_sqlite)
# Percorre os IDs
while id_author_start <= id_author_end:
  print('Reading the ID', id_author_start, end='...')
  author_data = get_researcher(id_author_start)
  if author_data == 'ID not found':
    print('ID not found')
  else:
    print('saving in the bank', end='...')
    #authors_mgp.append(author_data)
    author_data.save_to_db(db_sqlite)
    print('Saved!')

  id_author_start += 1
  #time.sleep(1)

## Consultado o banco

In [ ]:
df = pd.read_sql_query('''SELECT
                        r.researcher_id,
                        r.name AS researcher_name,
                        at.academic_title_id,
                        at.title AS academic_title,
                        at.dissertation,
                        at.institution,
                        at.year,
                        at.country,
                        GROUP_CONCAT(aat.advisor_id) AS advisor_ids  -- Lista os orientadores em uma única coluna
                    FROM academic_titles at
                    JOIN researchers r ON r.researcher_id = at.researcher_id
                    LEFT JOIN advisors_academic_titles aat ON at.academic_title_id = aat.academic_title_id
                    GROUP BY at.academic_title_id''', sqlite3.connect(db_sqlite))
sqlite3.connect(db_sqlite).close()

df.tail(10)

## Otimizando DF e novas colunas em um novo arquivo

In [ ]:
df_optimized = df.copy()

df_optimized["advisor_ids"] = df_optimized["advisor_ids"].apply(lambda x: x.split(",") if x else [])  # transforma os ids do orientadores em uma lista Python
df_optimized["year"] = pd.to_numeric(df_optimized["year"], errors="coerce")                           # converte o ano em número
df_optimized["num_advisors"] = df_optimized["advisor_ids"].apply(len)                                 # Contar quantos orientadores cada título teve e adiciona a coluna

# Calcular o número de alunos por pesquisador
tqdm.pandas(desc="Calculando número de alunos")
df_optimized["num_students"] = df_optimized["researcher_id"].progress_apply(lambda rid: len(get_advisor_and_student_ids(rid)[1]))

df_optimized.to_feather(db_mgp_optimized)

In [ ]:
df_optimized = pd.read_feather(db_mgp_optimized)

df_optimized